### Титаник.

Домашнее задание.

In [2]:
import numpy as np
import pandas as pd

In [158]:
from  matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

%config InlineBackend.figure_format = 'retina'
from pylab import rcParams
rcParams['figure.figsize'] = (9, 6)

from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

import xgboost
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials



###  Данные

Прочитаем тренировочные и тестовые данные.

In [4]:
train = pd.read_csv('titanic/train.csv')
test  = pd.read_csv('titanic/test.csv')

In [270]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
is_test        891 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [269]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
is_test        418 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


### Feature еngeneering

Создаем и описываем фичи.
Чтобы не дублировать операции для тренировочной тестовой выборки, объединим их:
- удалим целевой столбеци из train
- добавим флаг тестовой выборки
- проверим, что что поля совпадают.

In [301]:
# Прочитаем данные еще раз
train = pd.read_csv('titanic/train.csv')
test  = pd.read_csv('titanic/test.csv')

y_train = train.Survived
# Удалим целевую функцию из train
train.drop('Survived',axis=1,inplace=True)
# Пометим выборки
train['is_test'] = 0
test['is_test'] = 1
# Склеим
df = pd.concat([train,test])
# Заменим пол со строковой переменной на числовую
df['IsMale'] = df.Sex.replace({'male':1, 'female':0})
df = feature_1(df)

In [ ]:
df_have_age  = df[df.Age.notnull()].copy()
df_without_age = df[df.Age.isnull()].copy()
y_age = df_have_age.Age
df_have_age.drop(['Age','PassengerId','Name','Sex','Ticket','Cabin','is_test'],axis=1,inplace=True)
df_without_age.drop(['Age','PassengerId','Name','Sex','Ticket','Cabin','is_test'],axis=1,inplace=True)

df_have_age_d = pd.get_dummies(df_have_age, columns=['Pclass','Embarked'])
df_without_age_d = pd.get_dummies(df_without_age, columns=['Pclass','Embarked'])

columns = df_have_age_d.columns

imputer = Imputer(missing_values='NaN', strategy='mean', axis = 0, verbose=1, copy = True)
imputer.fit(df_have_age_d)

df_have_age_d_i = imputer.transform(df_have_age_d)
df_have_age_d_i = pd.DataFrame(df_have_age_d_i, columns=columns)

scaler = StandardScaler()
scaler.fit(df_have_age_d_i)
df_have_age_d_i_s = scaler.transform(df_have_age_d_i)
df_have_age_d_i_s = pd.DataFrame(df_have_age_d_i_s, columns = columns)


df_without_age_d_i_s = scaler.transform(imputer.transform(df_without_age_d))
df_without_age_d_i_s = pd.DataFrame(df_without_age_d_i_s, columns = columns)
age_clf = get_xgboost_best_model(df_have_age_d_i_s, y_age, 'r2')
#df_without_age_d_i_s.info()

pandas.core.series.Series

Давим фичу HaveCabin - проверим у кого указана каюта.

In [128]:
df['HaveCabin'] = df.Cabin.isnull()
df.HaveCabin.replace({True : 1, False : 0},inplace=True)

In [102]:
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_test,IsMale,HaveMrs,HaveMr,HaveCol,HaveCabin
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,1,0,1,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,0,1,0,0,0
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0,0,0,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,0,1,0,0,0
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,1,0,1,0,1


Все оставшивиеся текстовые поля удалим.

In [129]:
df.drop(['PassengerId','Name','Sex','Ticket','Cabin'],axis=1,inplace=True)

Поля - категогории разобьем на несколько столбцов

In [131]:
df_dummies = pd.get_dummies(df, columns=['Pclass','Embarked'])


Разделим тренировочную и тестовую выборку.

In [132]:
X_train = df_dummies[df_dummies.is_test == 0].drop('is_test',axis = 1)
X_test = df_dummies[df_dummies.is_test == 1].drop('is_test', axis = 1)

### Заполнение пустых значений. 

Пустые значения заполняем средними.
На основе обучающей выборки создаем imputer.

In [133]:
columns = X_train.columns
imputer = Imputer(missing_values='NaN', strategy='mean', axis = 0, verbose=1, copy = True)
imputer.fit(X_train)

Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=1)

In [134]:
X_train_imputed = imputer.transform(X_train)
X_train_imputed = pd.DataFrame(X_train_imputed, columns=columns)

### Нормировка значений

Для обучающей выборки создаем scaler - нормировщик.

In [135]:
scaler = StandardScaler()
scaler.fit(X_train_imputed)
X_train_i_s = scaler.transform(X_train_imputed)
X_train_i_s = pd.DataFrame(X_train_i_s, columns = columns)

 #### Заполняем пустые значения и нормируем тестовую выборку
 
Используем объекты imputer и scaler, обученные на обучающей выборке. Применяем эти объекты к тестовой выборке.

In [155]:
X_test_i_s = scaler.transform(imputer.transform(X_test))
X_test_i_s = pd.DataFrame(X_test_i_s, columns = columns)

In [264]:
best_xgboost_clf = get_xgboost_best_model(X_train_i_s, y_train)

Best score 1 (accuracy):  0.835016835017
Fitting 8 folds for each of 12 candidates, totalling 96 fits


[Parallel(n_jobs=1)]: Done  96 out of  96 | elapsed:   14.4s finished


Best score 2 (accuracy):  0.836139169473
Best score 3 (accuracy):  0.829405162738
Best score 4 (accuracy):  0.841750841751
Fitting 8 folds for each of 25 candidates, totalling 200 fits


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:   24.5s finished


Best score 5 (accuracy):  0.83950617284
Best score 6 (accuracy):  0.83950617284


In [266]:
clf = xgboost.XGBClassifier(**best_xgboost_clf)
clf.fit(X_train_i_s, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.9,
       gamma=0.5, learning_rate=0.025, max_delta_step=0, max_depth=3,
       min_child_weight=5, missing=None, n_estimators=400, nthread=-1,
       objective='binary:logistic', reg_alpha=1e-05, reg_lambda=0.01,
       scale_pos_weight=0.6229508196721312, seed=0, silent=True,
       subsample=0.7)

# HyperOpt

In [241]:
hp.quniform('max_depth', 1, 10, 1)

In [277]:
# в этой функции мы проверяем, как ведёт себя модель при заданных параметрах
def score(params):
    #print("Training with params : ")
    #print(params)
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = int(params['n_estimators'])
    clf = xgboost.XGBClassifier(**params)
    #accurs = []
    aucs = []
    # Для оценки качества используем KFold, который определили выше
    for train_idx, test_idx in cv.split(X_train_i_s):
        X_train_fold, X_test_fold = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_train_fold, y_test_fold = y_train.iloc[train_idx], y_train.iloc[test_idx]
        clf.fit(X_train_fold, y_train_fold)
        preds = clf.predict_proba(X_test_fold)
        auc = roc_auc_score(y_test_fold, preds[:, 1])
        aucs.append(auc)
        #accur = clf.score(X_test_fold, y_test_fold)
        #accurs.append(accur)
    auc = np.mean(aucs)
    result = {'loss':  auc, 'status': STATUS_OK}
    print(result)
    return result


# это наша главная функция, в которой мы задаём параметры
def optimize(trials):
    space = {
             'n_estimators' : hp.quniform('n_estimators', 100, 500, 50), # (название параметра, от, до, шаг)
             'learning_rate' : hp.quniform('learning_rate', 0.005, 0.03, 0.005),
             'max_depth' : hp.quniform('max_depth', 1, 10, 1),
             'min_child_weight' : hp.quniform('min_child_weight', 1, 6, 1),
             'subsample' : hp.quniform('subsample', 0.5, 1, 0.1),
             'gamma' : hp.quniform('gamma', 0.0, 0.5, 0.1),
             'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.1),
             'objective': 'reg:linear',
             'silent' : 1,
             'scale_pos_weight': hp.quniform('scale_pos_weight', 0.5, 2, 0.5),
             'reg_alpha': 0.0,
             'reg_lambda': hp.quniform('reg_lambda', 0.01, 0.02, 0.005),
             }

    best = fmin( score, space, algo=tpe.suggest, trials=trials, max_evals=1)
    print('Best: ')
    print(best)

#сюда будет записана
trials = Trials()

optimize(trials)

{'loss': 0.88408057022436615, 'status': 'ok'}
Best: 
{'colsample_bytree': 0.5, 'gamma': 0.4, 'learning_rate': 0.03, 'max_depth': 10.0, 'min_child_weight': 4.0, 'n_estimators': 300.0, 'reg_lambda': 0.01, 'scale_pos_weight': 1.0, 'subsample': 0.8}


In [278]:
best_params_7 = trials.best_trial

In [244]:
my_trials  = trials

In [245]:
my_max = 1
my_num = 0
for i in my_trials:
    tmp = i['result']['loss']
    if tmp <  my_max:
        my_num = i['tid']
        my_max = tmp
my_num

93

In [246]:
best_params_7 = trials.trials[my_num]['misc']['vals']
best_params_7
best_params_8 = best_params_7.copy()
best_params_8

{'colsample_bytree': [0.8],
 'gamma': [0.4],
 'learning_rate': [0.005],
 'max_depth': [1.0],
 'min_child_weight': [3.0],
 'n_estimators': [300.0],
 'reg_lambda': [0.01],
 'scale_pos_weight': [1.5],
 'subsample': [0.9]}

In [247]:
best_params_8['colsample_bytree'] = float(best_params_8['colsample_bytree'][0])
best_params_8['gamma'] = float(best_params_8['gamma'][0])
best_params_8['learning_rate'] = float(best_params_8['learning_rate'][0])
best_params_8['max_depth'] = int(best_params_8['max_depth'][0])
best_params_8['min_child_weight'] = int(best_params_8['min_child_weight'][0])
best_params_8['n_estimators'] = int(best_params_8['n_estimators'][0])
best_params_8['scale_pos_weight'] = float(best_params_8['scale_pos_weight'][0])
best_params_8['reg_lambda'] = float(best_params_8['reg_lambda'][0])
best_params_8['subsample'] = float(best_params_8['subsample'][0])
best_params_8

{'colsample_bytree': 0.8,
 'gamma': 0.4,
 'learning_rate': 0.005,
 'max_depth': 1,
 'min_child_weight': 3,
 'n_estimators': 300,
 'reg_lambda': 0.01,
 'scale_pos_weight': 1.5,
 'subsample': 0.9}

In [248]:
clf = xgboost.XGBClassifier(**best_params_8)
clf.fit(X_train_i_s,y_train)
clf.score (X_train_i_s, y_train)

0.78226711560044893

Создаем сабмит

In [249]:
#clf = xgboost.XGBClassifier(**best_params_8)
#clf.fit(X_train_i_s,y_train)
y_test = clf.predict(X_test_i_s)
submit = pd.DataFrame(test.PassengerId)
submit['Survived'] = y_test
submit.tail()
submit.to_csv('submit.csv', index = False)

In [282]:
def feature_1(df_src):
    df = df_src.copy()

    df['HaveMrs'] = df.Name.str.contains('mrs.', case=False)
    df.HaveMrs.replace({True:1, False : 0}, inplace=True)

    df['HaveMr'] = df.Name.str.contains('mr\.', case = False)
    df.HaveMr.replace({True : 1, False : 0},inplace=True)

    df['HaveRev'] = df.Name.str.contains('rev\.', case = False)
    df.HaveRev.replace({True : 1, False : 0},inplace=True)

    df['HaveMiss'] = df.Name.str.contains('miss\.', case = False)
    df.HaveMiss.replace({True : 1, False : 0},inplace=True)

    df['HaveDr'] = df.Name.str.contains('dr\.', case = False)
    df.HaveDr.replace({True : 1, False : 0},inplace=True)

    df['HaveMaster'] = df.Name.str.contains('master\.', case = False)
    df.HaveMaster.replace({True : 1, False : 0},inplace=True)

    df['HaveCol'] = df.Name.str.contains('col\.', case = False)
    df.HaveCol.replace({True : 1, False : 0},inplace=True)
    return df;

In [328]:
def get_xgboost_best_model(X_train,y_train, score_type, ones_ratio = 1):
    # посчитаем соотношение между классами
    #ones_ratio = y_train[y_train == 1].shape[0] * 1.0 / y_train[y_train == 0].shape[0] 
    """ 
    Шаг 1: Зафиксируем learning_rate и параметры дерева и подберём n_estimators¶

    Параметры:
    
    max_depth. Как указанов в таблице выше, обычно варьируется в интервале от 3 до 10 
        (но от задачи к задаче значения могут меняться). В качестве начального значения обычно используют 5
    min_child_weight. Если выборка сильно несбалансирована, то лучше выбрать значение "1". 
        Иначе лучше выбрать значение "2" и зафиксировать
    gamma. Обычно выставляют значение в интервале от 0 до 0.2 и фиксируют. 
        В дальнейшем этот параметр всегда можно затюнить отдельно
    subsample, colsample_bytree. Выставим 0.8 и зафиксируем. 
        Можно также проварьировать в интервале 0.5-0.9.
    scale_pos_weight. Выставляется в зафисимости от соотношения классов в выборке и фиксируется
    """
    param_grid = {
        # параметры ансамбля
        'n_estimators': [10, 30, 50, 100, 200, 400, 600, 1000],
        #'n_estimators': [10, 30, 50],
        'learning_rate': [0.1, ],
        
        # параметры дерева
        'max_depth': [5],
        'min_child_weight': [2],
        'gamma': [0.1],
        'subsample': [0.8],
        'colsample_bytree': [0.8],
        'scale_pos_weight': [ones_ratio],
        
        # параметры регуляризации
        'reg_alpha': [0.0],
        'reg_lambda': [1.0]
    }

    cv = KFold(n_splits=8, shuffle=True)

    clf = xgboost.XGBClassifier()
    gs = GridSearchCV(clf, param_grid, scoring=score_type, cv=cv, verbose=0)

    gs.fit(X_train, y_train)
    best_params_1 = gs.best_estimator_.get_params()
    print('Best score 1 ({0}):{1} '.format(score_type, gs.best_score_))
    #print('Best params: ', best_params_1)

    """
    Шаг 2. Подбираем параметры дерева

        max_depth - будем варьировать от 3 до 10 с шагом 2
        min_child_weight - от 1 до 6 с шагом 2

    """

    param_grid = {
        'max_depth': range(3, 10, 2),
        'min_child_weight': range(1, 6, 2)
    }

    clf = xgboost.XGBClassifier(**best_params_1) # в качестве отправной точки возьмём модель с наилучшими параметрами предыдущего шага

    gs = GridSearchCV(clf, param_grid, scoring=score_type, cv=cv, verbose=1)

    gs.fit(X_train, y_train)
    best_params_2 = gs.best_estimator_.get_params()
    print('Best score 2 ({0}):{1} '.format(score_type, gs.best_score_))

    #print('Best params: ', best_params_2)
    
    """
    Шаг 3. Подбираем gamma (критерий создания поддерева)
        gamma - от 0 до 0.5 с шагом 0.1
    """

    param_grid = {
        'gamma': [0.1*i for i in range(6)]
    }

    clf = xgboost.XGBClassifier(**best_params_2)
    gs = GridSearchCV(clf, param_grid, scoring=score_type, cv=cv, verbose=0)
    gs.fit(X_train, y_train)
    best_params_3 = gs.best_estimator_.get_params()
    print('Best score 3 ({0}):{1} '.format(score_type, gs.best_score_))
    #print('Best params: ', best_params_3)

    """
    Шаг 4. Затюним subsample и colsample_bytree¶

    subsample - от 0.5 до 1.0 с шагом 0.1
    colsample_bytree - от 0.5 до 1.0 с шагом 0.1
    """

    param_grid = {
        'subsample': [0.5 + 0.1*i for i in range(6)],
        'colsample_bytree': [0.5 + 0.1*i for i in range(6)]
    }

    clf = xgboost.XGBClassifier(**best_params_3)

    gs = GridSearchCV(clf, param_grid, scoring=score_type, cv=cv, verbose=0)

    gs.fit(X_train, y_train)
    best_params_4 = gs.best_estimator_.get_params()
    print('Best score 4 ({0}):{1} '.format(score_type, gs.best_score_))

    #print('Best params: ', best_params_4)
    
    """
    Шаг 5. Регуляризация
    reg_alpha [1e-5, 1e-2, 0.1, 1, 100]
    reg_lambda [1e-5, 1e-2, 0.1, 1, 100]
    """
    param_grid = {
        'reg_alpha': [1e-5, 1e-2, 0.1, 1, 100],
        'reg_lambda': [1e-5, 1e-2, 0.1, 1, 100]
    }

    clf = xgboost.XGBClassifier(**best_params_4)

    gs = GridSearchCV(clf, param_grid, scoring=score_type, cv=cv, verbose=1)

    gs.fit(X_train, y_train)
    best_params_5 = gs.best_estimator_.get_params()
    print('Best score 5 ({0}):{1} '.format(score_type, gs.best_score_))

    #print('Best params: ', best_params_5)
    
    """
    Шаг 6. Learning rate

    Чем меньше у нас n_estimators в ансамбле, тем быстрее нам нужно двигаться с каждым шагом 
    (добавлением нового классификатора), т.е. делать больший learning_rate. 
    Обычно learning rate варьируют так, чтобы произведение n_estimators x learning_rate оставалось инвариантным
    """
    best_params_6 = best_params_5.copy()
    clf = xgboost.XGBClassifier(**best_params_5)
    best_n_estimators = clf.get_params()['n_estimators'] # возьмём наилучшие значения n_estimators с предыдущего шага
    best_learning_rate = best_params['learning_rate'] # аналогичная запись
    invariant_composition = best_n_estimators * best_learning_rate
    n_estimators_range = [10, 30, 100, 200, 400, 600, 800, 1000]

    best_score = gs.best_score_ # возьмём наилучшее качество с предыдущего шага

    for n_estimators in n_estimators_range:
        learning_rate = invariant_composition / n_estimators
        clf.set_params(n_estimators=n_estimators, learning_rate=learning_rate)
        #aucs = []
        accurs = []
        for train_idx, test_idx in cv.split(X_train):
            X_train_fold, X_test_fold = X_train.iloc[train_idx], X_train.iloc[test_idx]
            y_train_fold, y_test_fold = y_train.iloc[train_idx], y_train.iloc[test_idx]
            clf.fit(X_train_fold, y_train_fold)
            #preds = clf.predict_proba(X_test_fold)
            #auc = roc_auc_score(y_test_fold, preds[:, 1])
            #aucs.append(auc)
            accur = clf.score(X_test_fold, y_test_fold)
            accurs.append(accur)
        accur = np.mean(accurs)
        if accur > best_score:
            best_n_estimators = n_estimators
            best_learning_rate = learning_rate
            best_score = accur
    
    best_params_6['n_estimators'] = best_n_estimators
    best_params_6['learning_rate'] = best_learning_rate

    print('Best score 6: ', best_score)
    
    return best_params_6